In [1]:
import openai as client
import json


# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#     model="gpt-4-1106-preview",
#     tools=functions
# )
assistant_id = "asst_3Dnr4jdvIfg2JsSWnJG04VVA"

In [2]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want you to help me with thie file"
        }
    ]
)
thread

Thread(id='thread_5dymuBwxkgXPq3ajZuuBbIjF', created_at=1702482798, metadata={}, object='thread')

In [ ]:
file = client.files.create(
    file=client.file_from_path("./files/chapter_one.txt"), purpose="assistants"
)
file

In [ ]:
client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="", file_ids=[file.id]
)

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id
)
run

In [ ]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")
        for annotation in message.content[0].text.annotations:
            print(f"Source: {annotation.file_citation}")


def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": function_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outputs = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id,
        tool_outputs=outputs
    )

In [ ]:
get_run(run.id, thread.id).status

In [ ]:
get_messages(thread.id)